<a href="https://colab.research.google.com/github/TaekGwonNam/2024.VAI/blob/main/%EC%8B%A4%EC%8A%B502_MLP%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np


In [ ]:
mnist_train=dset.MNIST("",train=True, transform=transforms.ToTensor(),
                       target_transform=None, download=True)
mnist_test=dset.MNIST("", train=False,transform=transforms.ToTensor(),
                      target_transform=None, download=True)


In [ ]:
print ("mnist_train길이:", len(mnist_train))
print ("mnist_test 길이:", len(mnist_test))

image, label = mnist_train.__getitem__(0)
print("image data 형태: ", image.size())
print ("label: ", label)

img = image.numpy()
plt.title("label: %d" %label )
plt.imshow(img[0], cmap='gray')
plt.show()

In [ ]:
batch_size = 1023
learning_rate = 0.01
numepoch = 400

In [ ]:
train_loader = torch.utils.data.DataLoader(mnist_train,
                                           batch_size=batch_size,
                                           shuffle=True, num_workers=2,
                                           drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,
                                           batch_size=batch_size,
                                           shuffle=False, num_workers=2,
                                           drop_last=True)

In [ ]:
n = 3
for i, [imgs, labels] in enumerate(test_loader):
  if i>5:
    break

  print ("[%d]" %i)
  print ("한번에 로드되는 데이터 크기:", len(imgs))

  for j in range(n):
    img=imgs[j].numpy()
    img = img.reshape((img.shape[1], img.shape[2]))

    plt.subplot(1, n, j+1)
    plt.imshow(img, cmap='gray')
    plt.title("label: %d" %labels[j] )
  plt.show()

In [ ]:
model = nn.Sequential(
    nn.Linear(28*28,255),
    nn.Sigmoid(),
    nn.Linear(256,128),
    nn.Linear(128,10)
)

In [ ]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))




def softmax(x):
  ex = np.exp(x)
  return ex / np.sum(ex, axis=1, keepdims=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class MyMLP(nn.Module):
    def __init__(self, n_input, n_hidden1, n_hidden2, n_output):
        super(MyMLP, self).__init__()
        self.fc1 = nn.Linear(n_input, n_hidden1)
        self.fc2 = nn.Linear(n_hidden1, n_hidden2)
        self.fc3 = nn.Linear(n_hidden2, n_output)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 이미지를 평탄화합니다
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:


def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0

    for j, [imgs, labels] in enumerate(dloader):
        imgs = imgs.view(imgs.size(0), -1)  # 이미지를 평탄화합니다
        img = Variable(imgs)
        label = Variable(labels)
        output = imodel(img)
        _, output_index = torch.max(output, 1)

        total += label.size(0)
        correct += (output_index == label).sum().float()

    print("Accuracy of Test Data: {}".format(100*correct/total))

ComputeAccr(test_loader, model)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms

# 데이터 로드
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
mnist_train = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
mnist_test = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

batch_size = 1024
learning_rate = 0.01
num_epochs = 10

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

# 이미지 확인
n = 3
for i, [imgs, labels] in enumerate(test_loader):
    if i > 5:
        break

    print(f"[{i}]")
    print("한번에 로드되는 데이터 크기:", len(imgs))

    for j in range(n):
        img = imgs[j].numpy()
        img = img.reshape((img.shape[1], img.shape[2]))
        plt.subplot(1, n, j+1)
        plt.imshow(img, cmap='gray')
        plt.title("label: %d" % labels[j])
    plt.show()

# 모델 정의
class MyMLP(nn.Module):
    def __init__(self, n_input, n_hidden1, n_hidden2, n_output):
        super(MyMLP, self).__init__()
        self.fc1 = nn.Linear(n_input, n_hidden1)
        self.fc2 = nn.Linear(n_hidden1, n_hidden2)
        self.fc3 = nn.Linear(n_hidden2, n_output)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 이미지를 평탄화합니다
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델 생성
model = MyMLP(28*28, 255, 128, 10)

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 모델 학습
for epoch in range(num_epochs):
    total_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(mnist_train)//batch_size, loss.item()))

# 학습된 모델의 정확도 계산
def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0

    for j, [imgs, labels] in enumerate(dloader):
        imgs = Variable(imgs)
        label = Variable(labels)
        output = imodel(imgs)
        _, output_index = torch.max(output, 1)

        total += label.size(0)
        correct += (output_index == label).sum().float()

    print("Accuracy of Test Data: %.2f" % (100*correct/total))

ComputeAccr(test_loader, model)


In [ ]:
mysum = 0
m = len(mnist_test)
cnt = 0

for i in range(m):
    image, label = mnist_test.__getitem__(i)
    output = model(image.unsqueeze(0))  # 배치 차원 추가

    if (i % 1000 == 0):
        img = image.numpy()
        pred_label = output.argmax().item()  # argmax() 수정
        plt.title("pred: %d, label: %d" %(pred_label, label))
        plt.imshow(img[0], cmap='gray')
        plt.show()

    cnt += 1
    mysum += (output.argmax().item() == label)

print("정확도: %.2f" % ((float(mysum) / cnt) * 100.0))


In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
num_epoch = 400
loss_func = nn.CrossEntropyLoss()

for i in range(num_epoch):
    running_loss = 0.0
    for j, [imgs, labels] in enumerate(train_loader):
        optimizer.zero_grad()

        imgs = imgs.view(imgs.size(0), -1)
        output = model(imgs)
        loss = loss_func(output, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    if i % 50 == 0:
        print("%d.." % i)
        ComputeAccr(test_loader, model)
        print("Epoch %d, Loss: %.4f" % (i, running_loss / len(train_loader)))


In [ ]:
ComputeAccr(test_loader, model)

In [ ]:
netname = './nets/mlp_weight.pkl'
torch.save(model, netname, )

#model = torch.load(netname)

In [ ]:
np.savez_compressed('./nets/mlp_weight.npz',
                    w1=w1, b1=b1,
                    w2=w2, b2=b2,
                    w3=w3, b3=b3)